# Combine CSVs For Processing
Combines the CSVs so all relevant data is on one line and the data is placed based on linking keys through the different CSVs

Everything from this markdown cell to the next one is just a sanity check to make sure the chunking for the CSVs is not giving an odd output. Don't run unless you have some reason to check.

# Combine with Dask

We use dask because it automatically chunks data

We need to determine what features we want to be able to determine which CSVs we need to merge

In [1]:
import dask.dataframe as dd

/home/cole/miniconda3/envs/capstone/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/cole/miniconda3/envs/capstone/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
# change to be reflective of your environment
data_dir = '/home/cole/Workspace/School/TestData/'

In [4]:
BACKUP_OBJECTS = dd.read_csv(data_dir + 'BACKUP_OBJECTS.csv', dtype = {'META_UPDATE': 'str', 'DEACDATE': 'str','ATTRLENGTH': 'float64','FLAGS': 'float64','STG_HINT': 'float64', 'OWNER' : 'str'})
AFBF = dd.read_csv(data_dir + 'AF_BITFILES.csv')
SDRO = dd.read_csv(data_dir + 'SD_RECON_ORDER.csv')
SS_POOLS = dd.read_csv(data_dir + 'SS_POOLS.csv')

In [5]:
df = BACKUP_OBJECTS.merge(AFBF, how='left', left_on ='OBJID', right_on= 'BFID')

In [6]:
df = df.merge(SDRO, how='left', left_on ='OBJID', right_on= 'OBJID' )

In [11]:
df.to_parquet(data_dir + 'merged_data/')